### Label evolution and conformal prediction as post-processing methods for network predictions

In [ ]:
''' In the context of this project:
This is an old file that experimented with some label evolution+conformal prediction features.
The implementation of conformal prediction is not accurate
as it does not use the quantile method to find the threshold q_hat.
However, it can be seen as a precursor to the project and some of the data visualization code is useful
- M'''

In [ ]:
import os
import numpy as np
import pandas as pd
import glob
import re
from scipy.special import softmax

from scipy.io import loadmat
from skimage.transform import resize
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# analyze the dataframes for training and validation from each epoch
# we start by grabbing the dataframes

#csv_folder = "../logs/Baseline_r2plus1d_all/csvs/"
csv_folder = "../logs/R2plus1d_all_pseudo_round1/csvs/"
train_csvs = glob.glob(f"{csv_folder}/train_*.csv")
val_csvs = glob.glob(f"{csv_folder}/val_*.csv")
data_root = "/workspace/datasets/Aortic_Stenosis/as_tom/round2/" # the original CSV which we can depart information from
annotations_df = pd.read_csv(os.path.join(data_root, 'annotations-all.csv'))

def extract_file_index(filename):
    # Use regular expression to find numerical part in the filename
    match = re.findall(r'\d+', filename)[-1]
    if match:
        numerical_part = int(match)
        return numerical_part
    else:
        # Return a default value or handle the case where no numerical part is found
        return None

# sort ascending by the numerical suffix for each file with regex search
def sort_by_epoch(files_list):
    indices = []
    for i in range(len(files_list)):
        index = extract_file_index(files_list[i])
        indices.append(index)
    new_array = [None] * len(indices)
    for j in range(len(indices)):
        new_array[indices[j]] = files_list[j]
    return new_array

print(sort_by_epoch(train_csvs))

train_df = [pd.read_csv(x) for x in sort_by_epoch(train_csvs)]
val_df = [pd.read_csv(x) for x in sort_by_epoch(val_csvs)]

# since train_df is ordered randomly, sort each dataframe by the filename
train_df = [df.sort_values(by=['filename']).reset_index() for df in train_df]
val_df = [df.sort_values(by=['filename']).reset_index() for df in val_df]

In [ ]:
def df_to_logits(df, sm=False):
    # take the outputs_x rows and conver them to array of (N, C)
    cols = [x for x in df.columns if 'outputs' in x]
    logits = df[cols].to_numpy()
    if sm:
        logits = softmax(logits, axis=1)
    return logits

train_probs = [df_to_logits(x, True) for x in train_df]
val_probs = [df_to_logits(x, True) for x in val_df]

In [ ]:
# show the trend-line of prediction F1 score for training and test set
def plot_f1_over_time(epoch_0_dataframe, all_probs):
    # epoch_0_dataframe is the dataframe results at epoch 0
    # probs_progression is the list of probabilities for each epoch
    gt = np.array(epoch_0_dataframe['y'])
    preds = [np.argmax(p, axis=1) for p in all_probs]
    f1s = [f1_score(gt, p, average='macro') for p in preds]
    fig, ax = plt.subplots(figsize=(12,4))
    t = np.arange(len(preds))
    ax.plot(t, f1s)
    plt.show()

def cf_over_time(epoch_0_dataframe, all_probs):
    gt = np.array(epoch_0_dataframe['y'])
    preds = [np.argmax(p, axis=1) for p in all_probs]
    [print(confusion_matrix(gt, p)) for p in preds]

plot_f1_over_time(train_df[0], train_probs)
plot_f1_over_time(val_df[0], val_probs)
cf_over_time(train_df[0], train_probs)
cf_over_time(val_df[0], val_probs)

### Label evolution

In [ ]:
def get_cine_thumbnail(filename):
    cine_original = loadmat(filename)["cine"]  # T_original xHxW
    T_original = len(cine_original)
    thumbnail = cine_original[T_original//2] # HxW
    thumbnail = resize(thumbnail, (224,224))  # HxW, Note range becomes [0,1] here
    return thumbnail

# show the progression of predictions for a specific example
def plot_pred_progression(example_id, epoch_0_dataframe, all_probs):
    # example id is the row number in the epoch_0_dataframe
    # epoch_0_dataframe is the dataframe results at epoch 0
    # probs_progression is the list of probabilities for each epoch
    correct_class = epoch_0_dataframe.iloc[example_id]['y']
    filename = epoch_0_dataframe.iloc[example_id]['filename']
    example_probs = np.array([all_probs[t][example_id] for t in range(len(all_probs))]) # TxC
    thumbnail = get_cine_thumbnail(filename)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4))
    t = np.arange(len(example_probs))
    ax1.imshow(thumbnail, cmap='gray')
    ax2.plot(t, example_probs)
    plt.suptitle(f"{filename}, correct class = {correct_class}")
    plt.legend(['0','1','2','3'])
    plt.show()

def plot_pred_progression_study_level(echo_id, epoch_0_dataframe, all_probs):
    # first find a list of images for a certain study
    # then plot the training progression for each
    matching_entries = annotations_df[annotations_df['Echo ID#']==echo_id]['path']
    if len(matching_entries) <= 0:
        print(f"Found 0 matches for {echo_id}, exiting")
        return
    print(f"Found {len(matching_entries)} matches for {echo_id}")
    
    for entry in matching_entries:
        # find the example id of each matching entry
        full_filename = os.path.join(data_root, entry)
        entry_index = epoch_0_dataframe[epoch_0_dataframe['filename']==full_filename].index[0]
        plot_pred_progression(entry_index, epoch_0_dataframe, all_probs)
        
print(annotations_df[annotations_df['split']=='train']['Echo ID#'].value_counts())
plot_pred_progression_study_level(197635, train_df[0], train_probs)
#plot_pred_progression(2, train_df[0], train_probs)

In [ ]:
# does it make sense to look at progressions for the validation set?
print(annotations_df[annotations_df['split']=='val']['Echo ID#'].value_counts())
plot_pred_progression_study_level(227407, val_df[0], val_probs)

In [ ]:
# we hypothesize that for noisy labels in the dataset,
# they are first wrong (pretty often), then eventually overfit to the correct answer
# we create a "surprisal" metric for how different the GT is from the prediction progression
def calculate_surprisal(epoch_0_dataframe, all_probs, count_after_t=0, count_before_t=-1):
    assert count_after_t < len(all_probs)
    if count_before_t != -1:
        assert count_before_t > count_after_t and count_before_t < len(all_probs)
    gt = np.array(epoch_0_dataframe['y'])
    N = len(all_probs[0])

    probs_after_t = all_probs[count_after_t:count_before_t]
    probs_sum = np.array(probs_after_t).sum(axis=0)
    surprisal = len(probs_after_t) - probs_sum[range(N), gt]
    smoothed_label = probs_sum / len(probs_after_t)
    return surprisal, smoothed_label

train_surprisal, train_smoothed = calculate_surprisal(train_df[0], train_probs, 0, -1)

In [ ]:
# get the top 10 surprisal values (and what their labels should be), visualize the training progression
topk = 10
top_k_indices = (-train_surprisal).argsort()[:topk]
print(top_k_indices)
for i in top_k_indices:
    print(train_df[0]['y'][i])
    print(train_smoothed[i])
for i in top_k_indices:
    plot_pred_progression(i, train_df[0], train_probs)

In [ ]:
# if we process the entire dataset this way, how "one-hot" are the labels?
N = len(train_smoothed)
gt = np.array(train_df[0]['y'])
smoothed_max = np.argmax(train_smoothed,axis=1)
n_unclean_labels = np.sum(gt!=smoothed_max)
print(n_unclean_labels)
plt.hist([train_smoothed[range(N), gt],train_smoothed[range(N), smoothed_max]], bins=49)
plt.legend(['original class', 'pseudo class'])
plt.title('Label certainty distribution')

In [ ]:
# add the smoothened labels back to the original dataframe
# first create a corespondence between the rows of the original dataframe and the rows of train_df
filenames = annotations_df['path']
pseudolabels = []
for j, entry in enumerate(list(filenames)):
    full_filename = os.path.join(data_root, entry)
    index_in_smoothed = train_df[0][train_df[0]['filename']==full_filename].index
    if len(index_in_smoothed) <= 0: # validation or test set, fill with one-hot
        pseudolabel = [0.0, 0.0, 0.0, 0.0]
    else:
        pseudolabel = train_smoothed[index_in_smoothed[0]]
    pseudolabels.append(pseudolabel)

pseudolabels = np.array(pseudolabels)
pseudolabels.shape

In [ ]:
output_pseudolabel_csv = False
if output_pseudolabel_csv:
    for c in range(pseudolabels.shape[1]):
        output_channel = f"pseudolabel_{c}"
        annotations_df[output_channel] = pseudolabels[:, c]
    annotations_df
    # save the annotations to a new location
    annotations_df.to_csv(os.path.join(csv_folder, 'annotations-all-with-pseudolabels.csv'))

### Conformal prediction

In [ ]:
# we are setting up the validation set as the calibration set, to be more kosher, you should use an unseen subset of training
# check the certainty distribution of validation set
def certainty_hist(y, preds, epoch=-1):
    # y is (N,) integer array of C classes
    # preds is (N, C) array
    pred_classes = np.argmax(preds,axis=1)
    pred_confidence = np.max(preds,axis=1)
    gt_confidence = preds[range(len(y)),y]
    print(f"Epoch {epoch}, F1 score = {f1_score(y, pred_classes, average='macro')}")
    fig, ax = plt.subplots(figsize=(8,3))
    ax.hist([gt_confidence, pred_confidence], bins=49)
    plt.legend(['GT', 'argmax'])
    plt.title(f"Epoch {i} label certainty distribution")
    plt.show()

for i in range(len(val_df)):
    y = val_df[i]['y']
    preds = val_probs[i]
    certainty_hist(y, preds, i)

In [ ]:
# run conformal prediction on a select epoch to return the prediction sets
def conformal_prediction(y, preds, desired_accuracy=0.9):
    # y is (N,) integer array of C classes
    # preds is (N, C) array
    # desired_accuracy is equivalent to 1-alpha in CP literature
    N = len(y)
    pred_classes = np.argmax(preds,axis=1)
    pred_confidence = np.max(preds,axis=1)
    gt_confidence = preds[range(N),y]
    # calculate the conformal score s as a general imprecise uncertainty measure (can be many ways, this is a simple way)
    s = 1 - gt_confidence
    # find the q level
    q_level = np.quantile(s, desired_accuracy)
    adj_q_level = np.ceil((N+1) * desired_accuracy)/N
    # create sets based on the q-level
    cutoff = 1 - adj_q_level
    conformal_set = []
    for i in range(N):
        conformal_set.append(preds[i] >= cutoff)
    return np.array(conformal_set)

y = val_df[12]['y']
preds = val_probs[12]
cs = conformal_prediction(y, preds)

In [ ]:
# test the coverage of conformal prediction
def coverage_test(y, preds_cover):
    # y is (N,) integer array of C classes
    # preds_cover is (N, C) boolean array indicating the cover
    correct = []
    for i in range(len(y)):
        if preds_cover[i, y[i]]:
            correct.append(True)
        else:
            correct.append(False)
    return np.array(correct)

correctness = coverage_test(y, cs) 
print(np.sum(correctness)/len(y)) # should be around 0.9

In [ ]:
def class_conditional_coverage_test(y, preds_cover, num_classes=4):
    # returns an accuracy for each class
    class_cond_acc = np.zeros(num_classes)
    for c in range(num_classes):
        mask = y == c
        preds_c = preds_cover[mask]
        N_c = len(preds_c)
        correct = 0
        for i in range(N_c):
            if preds_c[i,c]:
                correct += 1
        class_cond_acc[c] = correct/N_c
    return class_cond_acc
correctness = class_conditional_coverage_test(y, cs, 4) 
correctness

In [ ]:
# check the cardinality of the conformal prediction
def cp_cardinality(y, preds_cover, preds):
    cardinality = np.sum(preds_cover, axis=1)
    correct = coverage_test(y, preds_cover)
    card_correct = cardinality[correct]
    card_incorrect = cardinality[~correct]
    
    c_amount, c_bins = np.histogram(card_correct, bins=np.arange(5)+0.5)
    n_amount, _ = np.histogram(card_incorrect, bins=np.arange(5)+0.5)
    card_acc = c_amount / (c_amount + n_amount + 1e-9)
    print(f"Number of correct preds of size 1/2/3/4: {c_amount}")
    print(f"Number of incorrect preds of size 1/2/3/4: {n_amount}")
    print(f"Accuracy of preds of size 1/2/3/4: {card_acc}")

    top1_accs = []
    pred_classes = np.argmax(preds, axis=1)
    for i in np.arange(4)+1:
        gt_w_card_i = y[cardinality==i]
        pred_w_card_i = pred_classes[cardinality==i]
        acc_w_card_i = np.sum(gt_w_card_i == pred_w_card_i) / (len(gt_w_card_i) + 1e-9)
        top1_accs.append(acc_w_card_i)
    print(f"Top-1 acc of preds of size 1/2/3/4: {top1_accs}")

cp_cardinality(y, cs, preds)

In [ ]:
# run conformal prediction on each epoch
f1s = []
coverages = []
for i in range(len(val_df)):
    y = np.array(val_df[i]['y'])
    preds = val_probs[i]
    pred_classes = np.argmax(preds,axis=1)
    print(f"Epoch {i}")
    print(f"Original classifier F1 = {f1_score(y, pred_classes, average='macro')}")
    f1s.append(f1_score(y, pred_classes, average='macro'))
    cs = conformal_prediction(y, preds)
    correctness = coverage_test(y, cs)
    print(f"Conformal coverage accuracy = {np.sum(correctness)/len(y)}")
    coverages.append(np.sum(correctness)/len(y))
    correctness = class_conditional_coverage_test(y, cs, num_classes=4)
    print(f"Class-conditional coverage accuracy = {correctness}")
    cp_cardinality(y, cs, preds)
    
plt.plot(range(len(val_df)), f1s)
plt.plot(range(len(val_df)), coverages)
plt.plot(range(len(val_df)), np.full(shape=len(val_df), fill_value=0.9))
plt.title('Label evolution: validation set top-1 F1 vs coverage over training')
plt.legend(['top-1 F1 score', 'coverage acc.', 'theoretical coverage acc.'])
plt.show()

In [ ]:
# problems so far
# 1. class-conditional coverage - can we have some CP algo with better guarantee for ea. class?
# 1a. review RAPS algorithm
# 2. cardinality relation w. difficulty - is it possible for low-cardinality sets to generally be more accurate?